<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/tps_march_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1. libraries and dataset

In [ ]:
!pip install category_encoders

In [ ]:
!pip install catboost

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [34]:
from sklearn.metrics import roc_auc_score

In [23]:
traindata = pd.read_csv("/content/drive/MyDrive/data/kaggle_tabplayground_mar/train.csv", index_col=0)
testdata = pd.read_csv("/content/drive/MyDrive/data/kaggle_tabplayground_mar/test.csv", index_col=0)
submission = pd.read_csv("/content/drive/MyDrive/data/kaggle_tabplayground_mar/sample_submission.csv", index_col=0)

In [24]:
traindf = traindata.copy()
testdf = testdata.copy()

#2. encoding

In [25]:
traindf.drop(columns=['cat5', 'cat13', 'cont2', 'cont7', 'cont10'], inplace=True)
testdf.drop(columns=['cat5', 'cat13', 'cont2', 'cont7', 'cont10'], inplace=True)

In [11]:
numcols = [col for col in traindf.columns if traindf[col].dtype != 'object']
catcols = [col for col in traindf.columns if col not in numcols]

In [13]:
for col in catcols:
    print(col, "\t", traindf[col].nunique())

cat0 	 2
cat1 	 15
cat2 	 19
cat3 	 13
cat4 	 20
cat6 	 16
cat7 	 51
cat8 	 61
cat9 	 19
cat10 	 299
cat11 	 2
cat12 	 2
cat14 	 2
cat15 	 4
cat16 	 4
cat17 	 4
cat18 	 4


In [30]:
catcols2 = ['cat0', 'cat11', 'cat12', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']
catcols3 = [col for col in catcols if col not in catcols2]

In [36]:
X = traindf.drop(columns=['target'])
y = traindf['target']

In [27]:
from category_encoders.one_hot import OneHotEncoder
onehot = OneHotEncoder(cols = catcols2, return_df=True, handle_unknown=0, drop_invariant=True)
X = onehot.fit_transform(X)
testdf = onehot.transform(testdf)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [32]:
kf = StratifiedKFold(shuffle=True, random_state=42)

In [33]:
from category_encoders.target_encoder import TargetEncoder

In [46]:
ind = kf.split(X=np.zeros(300000), y=y)
scores = []

for tr_ind, val_ind in ind:
    X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
    y_tr, y_val = y.iloc[tr_ind], y.iloc[val_ind]

    enc = TargetEncoder(cols=catcols3, return_df=True, handle_unknown=y.mean())
    X_tr = enc.fit_transform(X_tr, y_tr)
    X_val = enc.transform(X_val)

    lgbm = LGBMClassifier()
    lgbm.fit(X_tr, y_tr)
    pred = lgbm.predict_proba(X_val)[:, 1]
    scores.append(roc_auc_score(y_val, pred))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: Future

In [47]:
scores

[0.8876715063004289,
 0.888103496894598,
 0.8894566805749728,
 0.8881404017678868,
 0.8899232154608159]

In [49]:
enc = TargetEncoder(cols=catcols3, return_df=True, handle_unknown=y.mean())
X = enc.fit_transform(X, y)
testdf = enc.transform(testdf)
lgbm.fit(X, y)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# Submission

In [50]:
submission['target'] = lgbm.predict_proba(testdf)[:, 1]

In [52]:
submission.to_csv("submission.csv")